In [1]:
from dataController import DataController

data_path = '../data/Dados-Processo-tratados/todos_ensaios.csv'
data_controller = DataController(data_path)
data_controller.df_norm.head()

,time,sensor_PT100_1,sensor_PT100_3,sensor_PT100_4,sensor_humidity_1,sensor_TH_1,sensor_humidity_2,sensor_TH_2,product_humidity,ensaio,sensor_PT100_2
0,-1.647032,-1.781324,-0.594383,-0.783427,1.764704,-1.852509,0.458340,-1.334653,0.009587,-1.555791,-2.441895
1,-1.141096,-1.839644,-0.449604,-0.701485,1.818334,-1.920969,0.962101,-1.087991,0.916248,-1.555791,-2.411608
2,-0.630908,-1.817214,-0.453740,-0.681759,1.831141,-1.937269,1.128749,-0.961792,1.295651,-1.555791,-2.383582
3,-0.120720,-1.862075,-0.438573,-0.683276,1.875966,-1.969869,1.161824,-0.881483,1.906601,-1.555791,-2.370473
4,0.389467,-1.821700,-0.457877,-0.698450,1.892775,-1.920969,1.184722,-0.902516,0.743285,-1.555791,-2.353295


In [5]:
interest_column = 'product_humidity'
columns_to_drop = ['ensaio']
columns_to_drop.append(interest_column)

X = data_controller.df_norm.drop(columns=columns_to_drop).reset_index(drop=True)
y = data_controller.df[interest_column]

display(X.head())
display(y.head())

,time,sensor_PT100_1,sensor_PT100_3,sensor_PT100_4,sensor_humidity_1,sensor_TH_1,sensor_humidity_2,sensor_TH_2,sensor_PT100_2
0,-1.647032,-1.781324,-0.594383,-0.783427,1.764704,-1.852509,0.458340,-1.334653,-2.441895
1,-1.141096,-1.839644,-0.449604,-0.701485,1.818334,-1.920969,0.962101,-1.087991,-2.411608
2,-0.630908,-1.817214,-0.453740,-0.681759,1.831141,-1.937269,1.128749,-0.961792,-2.383582
3,-0.120720,-1.862075,-0.438573,-0.683276,1.875966,-1.969869,1.161824,-0.881483,-2.370473
4,0.389467,-1.821700,-0.457877,-0.698450,1.892775,-1.920969,1.184722,-0.902516,-2.353295


8     10.92
9     14.17
10    15.53
11    17.72
12    13.55
Name: product_humidity, dtype: float64